First, some parameters:
- Make sure that these are synchronized with the parameters from generate_main.ipynb if they have the same name

In [14]:
stack_size : int = 2e9 # 2 Gb sounds good for now

thread_init_function_name : str = "init_thread"

In [15]:
from math import *

In [16]:
with open("main.c", 'w') as file:

    #
    # HEADERS 
    #
    file.write("#include <stdio.h>\n")
    file.write("#include <ctype.h>\n")
    file.write("#include <stdint.h>\n")
    file.write("#include <inttypes.h>\n")
    file.write("#include <stdbool.h>\n")
    file.write("#include <string.h>\n")
    file.write("#include <pthread.h>\n\n")

    file.write('#include "src/primary_thread.h"\n\n')

    file.write("#define STDIN_BUFFER_SIZE 500 * sizeof(char)\n\n")

    #
    # BEGIN MAIN
    #
    file.write("int main()\n{\n")
               
    #
    # DEFINE VARIABLES THAT WE WANT TO BE ABLE TO READ 
    #

    file.write("\tchar input[STDIN_BUFFER_SIZE];\n")


    #
    # INIT PRIMARY THREAD
    #

    file.write(f"""
    memset(input, 0, STDIN_BUFFER_SIZE);

    int rc = 0;
    pthread_t thread;
    pthread_attr_t attr;
               
    pthread_mutex_t thread_init_mutex = PTHREAD_MUTEX_INITIALIZER;
    pthread_cond_t thread_init_cond = PTHREAD_COND_INITIALIZER;
    bool thread_is_initialized = false;
               
    ThreadArgs args = {{
        .thread = &thread,
        .thread_init_mutex = &thread_init_mutex,
        .thread_init_cond = &thread_init_cond,
        .thread_is_initialized = &thread_is_initialized,
        .attr = &attr
    }};
    
    rc = pthread_attr_init(&attr);
    if (rc != 0)
    {{
        fprintf(stderr, "main() - Err [0] - code: %u", rc);
        cleanup_thread(args, false, false);
        return 1;
    }}
    rc = pthread_attr_setstacksize(&attr, {floor(stack_size)});
    if (rc != 0)
    {{
        fprintf(stderr, "main() - Err [1] - code: %u", rc);  
        cleanup_thread(args, false, false);
        return 1;
    }}
    rc = pthread_create(&thread, &attr, {thread_init_function_name}, (void*)&args);
    if (rc != 0)
    {{
        fprintf(stderr, "main() - Err [2] - code: %u", rc);
        cleanup_thread(args, false, false);
        return 1;
    }}

    rc = pthread_mutex_lock(&thread_init_mutex);
    if (rc != 0)
    {{
        fprintf(stderr, "main() - Err [3] - code: %u", rc);
        cleanup_thread(args, true, false);
        return 1;
    }}

    while (!thread_is_initialized) 
    {{
        pthread_cond_wait(&thread_init_cond, &thread_init_mutex);  // Wait for the thread to finish initializing before continuing

        if(!thread_is_initialized)
        {{
            fprintf(stderr, "main() - Err [4] - code: %u", rc);
            cleanup_thread(args, true, false);
            return 1;
        }}
    }}

    rc = pthread_mutex_unlock(&thread_init_mutex);
    if (rc != 0)
    {{
        fprintf(stderr, "main() - Err [5] - Thread was not successfully initialized.");
        cleanup_thread(args, true, false);
        return 1;
    }}

""")

    #
    # TEST FOR BASIC FUNCTIONALITY
    #

    ###
    ###
    ###

    #
    # CONTROL LOOP
    #

    ###
    ###
    ###
        
    #
    # END MAIN
    #
    file.write("\tcleanup_thread(args, true, true);\n\n")

    file.write("\treturn 0;\n")

    file.write("};\n")